In [1]:
!pip install langchain

In [2]:
!pip install sentence_transformers

In [3]:
!pip install chromadb

In [4]:
import csv
import os
import sys
from typing import List, Dict

from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import CSVLoader

In [5]:
CHROMADB_DIR = "chromadb"
# EMBEDDINGS_MODEL = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
EMBEDDINGS_MODEL = "intfloat/multilingual-e5-large"
MODEL_FILE_NAME = "ggml-model-q4_1.bin"
MODEL_REPO = "IlyaGusev/saiga_13b_lora_llamacpp"

In [6]:
def populate_db() -> None:
    global CHROMADB_DIR, EMBEDDINGS_MODEL

    text_col_name = "text" if len(sys.argv) == 1 else sys.argv[1]
    raw_docs = []
    print(text_col_name)
    data_dir = "."
    for file_name in os.listdir(data_dir):
        if ".csv" not in file_name:
            continue
        csv_path = os.path.join(data_dir, file_name)
        loader = CSVLoader(file_path=csv_path)
        raw_docs.extend(loader.load())

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=250,  # min: 50, max: 2000
        chunk_overlap=30,  # min: 0, max: 500,
    )
    Chroma.from_documents(
        text_splitter.split_documents(raw_docs),
        HuggingFaceEmbeddings(model_name=EMBEDDINGS_MODEL),
        persist_directory=CHROMADB_DIR,
    ).persist()


def __find_similar(query: str) -> List[Document]:
    return Chroma(
        persist_directory=CHROMADB_DIR,
        embedding_function=HuggingFaceEmbeddings(
            model_name=EMBEDDINGS_MODEL
        ),
    ).similarity_search(query)

In [7]:
populate_db()

-f


In [17]:
__find_similar('Какие дополнительно документы должен предоставить заявитель по услуге "Предоставлять дополнительную меру социальной поддержки в виде единовременной компенсационной выплаты при рождении ребенка (усыновлении в возрасте до шести месяцев) для приобретения предметов детского ассортимента и продуктов детского питания"?')

[Document(page_content='QUESTION: Какие дополнительно документы должен предоставить заявитель по услуге "Предоставлять дополнительную меру социальной поддержки в виде единовременной компенсационной выплаты при рождении ребенка (усыновлении в возрасте до шести месяцев) для', metadata={'row': 37, 'source': './train_dataset_Датасет.csv'}),
 Document(page_content='QUESTION: Какие документы предоставить при обращении представителя по услуге "Предоставлять дополнительную меру социальной поддержки в виде единовременной компенсационной выплаты при рождении ребенка (усыновлении в возрасте до шести месяцев) для', metadata={'row': 31, 'source': './train_dataset_Датасет.csv'}),
 Document(page_content='QUESTION: Какие заявления брать у заявителя по услуге "Предоставлять дополнительную меру социальной поддержки в виде единовременной компенсационной выплаты при рождении ребенка (усыновлении в возрасте до шести месяцев) для приобретения предметов', metadata={'row': 33, 'source': './train_dataset_Датас

In [10]:
import pandas as pd

In [11]:
df = pd.read_csv('train_dataset_Датасет.csv')

In [15]:
df.iloc[37, 0]

'Какие дополнительно документы должен предоставить заявитель по услуге "Предоставлять дополнительную меру социальной поддержки в виде единовременной компенсационной выплаты при рождении ребенка (усыновлении в возрасте до шести месяцев) для приобретения предметов детского ассортимента и продуктов детского питания"?'

In [16]:
df.iloc[37, 1]

'1. Справка, подтверждающая постановку на медицинский учет по беременности в срок до 20 недель (включительно). В инживидульном случае: справка, подтверждающая постановку на учет по беременности в срок до 20 недель (включительно) женщины, родившей ребенка (суррогатной матери)\nИ\nдокумент, выданный медицинской организацией и подтверждающий факт получения согласия женщины, родившей ребенка (суррогатной матери), на запись родителями ребенка лиц, указанных в свидетельстве о рождении ребенка.  2. Документы, содержащие данные органов регистрационного учета заявителя и ребенка (детей) по месту жительства в Санкт-Петербурге. 3. Свидетельство о рождении ребенка, выданное органами ЗАГС РФ;\n\n- свидетельства о рождении предыдущих детей, выданные органами ЗАГС РФ;\n\n- свидетельство о смерти ребенка (в случае смерти предыдущего ребенка), выданное органами ЗАГС РФ;\n\n- документы об установления над ребенком опеки (в отношении детей, находящихся под опекой или попечительством) - выданные на террит

In [19]:
df.iloc[31, 1]

'1. Документ, удостоверяющий личность доверенного лица по доверенности  2. Нотариально удостоверенная, приравненная к нотариально удостоверенной или удостоверенная организацией, в которой доверитель работает или учится, или администрацией стационарного лечебного учреждения, в котором он находится на излечении. Или простая письменная доверенность'